# arguments

This module is meant to add all the support functions for *arguments* and *inout* parsing.

In [ ]:
#| default_exp inout

In [ ]:
from fastcore.test import *

## Find support function

`optionalFind` let you search a string in forward or reverse order using one or more *templates* (aka: reference sub-strings).

It returns the *first* occurrence that is the leftmost in forward direction and rightmost in reverse direction.
The occurrence returned is a tuple `(position,template)` where:
+ `position`: is the position inside the string.
+ `templace`: is the copy of the template found. We need this to understand where that string finishes.

In [ ]:
#| export
def optionalFind(x,cc,reverse=False):
    if isinstance(cc,str): cc = [cc] # listify
    t = [(x.rfind(c) if reverse else x.find(c),c) for c in cc]
    t = [(f,c) for f,c in t if f != -1]
    if len(t)==0: return None
    return max(t) if reverse else min(t)

In [ ]:
# Single
test_eq( optionalFind('abc','a')[0] , 0)
test_eq( optionalFind('abc','c')[0] , 2)
test_eq( optionalFind('abc','bc')[0] , 1)
test_eq( optionalFind('abc','z'), None)
test_eq( optionalFind('abcdefghiab','a',reverse=True)[0] , 9)
test_eq( optionalFind('abc','bc',reverse=True)[0] , 1)

# Multi
test_eq( optionalFind('abcdefghiab',['aa','fg','ia'],reverse=False)[0] , 5)
test_eq( optionalFind('abcdefghiab',['aa','fg','ia'],reverse=True)[0] , 8)
test_eq( optionalFind('abcdefghiab',['aa','ia','fg'],reverse=True)[0] , 8)

## Character level utility

In [ ]:
#| export
def count_char(x,c):
    # Count how many times c appears in x
    return sum(map(lambda y: y==c,x))

In [ ]:
test_eq( count_char('abacda','a') , 3 )
test_eq( count_char('abacda','b') , 1 )
test_eq( count_char('abacda','z') , 0 )

In [ ]:
#| export
def count_delta(x,a='(',b=')'):
    # Return the difference between the count of "a" and "b".
    return count_char(x,a) - count_char(x,b)

In [ ]:
test_eq(count_delta('asd(asd)'),0)
test_eq(count_delta('asd(asd'),1)
test_eq(count_delta('asd(a(sd'),2)
test_eq(count_delta('asd(a(sd)))'),-1)

## String split utility

This is just a bit of syntactic sugar

In [ ]:
#| export
def split_and_strip(x,splitter):
    t = [t.strip() for t in x.split(splitter)]
    if t==['']: return []
    return t

In [ ]:
test_eq(split_and_strip('a,b,c',','),['a','b','c'])
test_eq(split_and_strip(' a, b,c ',','),['a','b','c'])
test_eq(split_and_strip('',','),[])

## Inout util

`inout` is the *name* of the portion of the complete arguments string that refers to input and output parameters.

For example in this magick line: `%%testcell noglobals (aaa,bbb) ->(ccc)` 
+ *raw_arguments*: `noglobals `
+ *inout*: `(aaa,bbb) ->(ccc)`

In [ ]:
#| export
def process_inout(x,splitter='->'):
    if x is None: return None
    t = split_and_strip(x,splitter)
    for v,c,n in zip(t,map(count_delta,t),map(lambda s: count_char(s,'('),t)): 
        if n>1: raise ValueError(f'Too much parenthesis on "{v}"')
        if c>0: raise ValueError(f'Missing closing parenthesis on "{v}"')
    t = [x[1:-1] for x in t]
    if len(t)==0: raise ValueError('No groups available')
    if len(t)>2: raise ValueError(f'You shouold have only one "{splitter}" symbol')
    if len(t)==1: return split_and_strip(t[0],','),[]
    if len(t)==2: return split_and_strip(t[0],','),split_and_strip(t[1],',')

In [ ]:
test_eq(process_inout(None,splitter='->'),None)

test_eq(process_inout('(a,b,cc)->(d,ee)',splitter='->'),(['a','b','cc'],['d','ee']))
test_eq(process_inout(' (a,b,cc)  -> (d,ee) ',splitter='->'),(['a','b','cc'],['d','ee']))
test_eq(process_inout(' (a,b,cc)  -> () ',splitter='->'),(['a','b','cc'],[]))
test_eq(process_inout(' (a,b,cc)  ',splitter='->'),(['a','b','cc'],[]))
test_eq(process_inout('()->(a,b,cc)  ',splitter='->'),([],['a','b','cc']))
test_eq(process_inout('->(a,b,cc)  ',splitter='->'),([],['a','b','cc']))

test_fail(lambda: process_inout('(a,b,cc)(d,ee)'), contains='Too much parenthesis')
test_fail(lambda: process_inout('(a,b,cc) (d,ee)'), contains='Too much parenthesis')
test_fail(lambda: process_inout('(a,b,cc) (->d,ee)'), contains='Too much parenthesis')
test_fail(lambda: process_inout('(a,b,cc->)(d,ee)'), contains='Missing closing parenthesis')
test_fail(lambda: process_inout('(a,b,cc) - > (d,ee)'), contains='Too much parenthesis')
test_fail(lambda: process_inout('(a,b,cc) ? (d,ee)'), contains='Too much parenthesis')

In [ ]:
#| export
def separate_args_and_inout(x):
    if x is None: return None
    if (start_t := optionalFind(x,['(','->'],reverse=False)) and (end_t := optionalFind(x,[')','->'],reverse=True)):
        start,_ = start_t
        end, c = end_t
        length = len(c)
        return x[:start]+x[end+length:],x[start:end+length]
    return x,None

In [ ]:
test_eq(separate_args_and_inout(None),None)

test_eq(separate_args_and_inout(''),['',None])
test_eq(separate_args_and_inout('verbose'),['verbose',None])
test_eq(separate_args_and_inout('dryrun verbose'),['dryrun verbose',None])

test_eq(separate_args_and_inout('dryrun verbose (a,b)'),['dryrun verbose ','(a,b)'])
test_eq(separate_args_and_inout('dryrun verbose (a,b) -> (c,d) '),['dryrun verbose  ','(a,b) -> (c,d)'])
test_eq(separate_args_and_inout('dryrun verbose  () -> (c,d) '),['dryrun verbose   ','() -> (c,d)'])
test_eq(separate_args_and_inout('dryrun verbose (a,b) -> ()'),['dryrun verbose ','(a,b) -> ()'])
test_eq(separate_args_and_inout('dryrun verbose (a,b)  '),['dryrun verbose   ','(a,b)'])

test_eq(separate_args_and_inout('dryrun  (a,b)  verbose '),['dryrun    verbose ','(a,b)'])
test_eq(separate_args_and_inout('dryrun  (a,b) -> (c,d) verbose'),['dryrun   verbose','(a,b) -> (c,d)'])
test_eq(separate_args_and_inout('dryrun () -> (c,d) verbose'),['dryrun  verbose','() -> (c,d)'])
test_eq(separate_args_and_inout('dryrun () -> (c,d)verbose'),['dryrun verbose','() -> (c,d)'])
test_eq(separate_args_and_inout('dryrun() -> (c,d) verbose'),['dryrun verbose','() -> (c,d)'])
test_eq(separate_args_and_inout('dryrun  (a,b)->() verbose'),['dryrun   verbose','(a,b)->()'])
test_eq(separate_args_and_inout('dryrun (a, b)  verbose'),['dryrun   verbose','(a, b)'])

test_eq(separate_args_and_inout('dryrun -> (c,d)verbose'),['dryrun verbose','-> (c,d)'])
test_eq(separate_args_and_inout('dryrun (c,d)->verbose'),['dryrun verbose','(c,d)->'])

## Consume inout

In [ ]:
#| export
def validate_and_update_inputs(inputs:list,state:dict,)->dict:
    s = set(inputs)
    ret = {}
    for k in inputs:
        if k not in state: raise ValueError(f'Unable to find object "{k}" in current state')
        ret[k] = state[k]
    return ret

In [ ]:
test_eq( validate_and_update_inputs('a',{'a':1, 'b':2}) , {'a':1} )
test_fail(lambda: validate_and_update_inputs('a',{'b':1}) , contains='Unable' )

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()